# Detailed Pion Correlator

This notebook is to help explain each step of the CorrelationMatrix.run function.

In [1]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

In [30]:
from PyCorrelationMatrixManager.correlation_matrix import CorrelationMatrix
from WickContractions.laph.diagram import LDiagram
from make_ops import *

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))

creation_ops=[create_pion_op()]
annihilation_ops=[annihilate_pion_op()]

pprint(creation_ops[0])
pprint(annihilation_ops[0])

import copy

cmat = CorrelationMatrix(creation_ops, annihilation_ops, 
                        dts=[0], t0s=[0], cfg=0)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

First we perform the wick contractions on the correlation matrix, this fills all of the Correlator objects with an array of Diagram objects, according to Wick's theorem.

In [31]:
cmat.contract()
for d in cmat.correlators[0].diagrams:
    pprint(d)

<IPython.core.display.Math object>

Next we change the Diagram objects to LDiagram objects, or LapH diagrams, using the following rules

$$
    D^{-1}_u(t_f,x_1\mid t_i,x_0)_{\substack{s_3\\c_3}\substack{s_0\\c_0}} \rightarrow V^*(t_f,x_1)_{c_3l_3}\tau(t_f,x_1\mid t_i,x_0)_{\substack{s_3\\l_3}\substack{s_0\\c_0}} V(t_i,x_0)_{c_0,l_0}
$$
which is the same result we would've gotten if we applied the LapH smearing operator directly to the quarks.

In [32]:
for i,d in enumerate(copy.deepcopy(cmat.correlators[0].diagrams)):
    cmat.correlators[0].diagrams[i]=LDiagram(d)
    
for d in cmat.correlators[0].diagrams:    
    pprint(d)

<IPython.core.display.Math object>

Now we combine the eigenvectors and color functions into the functions

$$
    m(t,x)_{l_0l_1}=\delta_{c_0c_1}V(t,x)_{c_0l_0}V^*(t,x)_{c_1l_1} \\
    b(t,x)=\epsilon_{c_0c_1c_2}V(t,x)_{c_0 l_0}V(t,x)_{c_1 l_1}V(t,x)_{c_2 l_2} \\
    b^*(t,x) = \epsilon_{c_0c_1c_2}V^*(t,x)_{c_0 l_0}V^*(t,x)_{c_1 l_1}V^*(t,x)_{c_2 l_2}
$$

In [33]:
for d in cmat.correlators[0].diagrams:
    d.create_m_blocks()
    d.create_b_blocks()

for d in cmat.correlators[0].diagrams:
    pprint(d)

<IPython.core.display.Math object>

Now we combine the meson and baryon functions into hadron functions by adding in gamma matrices with the same spin sub indices as the eigenvector sub indices.  We also combine spin and eigenvector indices into a compound index 

$i=s_iN_v + l_i$

$$
    M(t,x,\gamma)_{01}=M(t,x,\gamma)_{s_0l_0,s_1l_1} = m(t,x)_{l_0l_1}\gamma_{s_0s_1} \\
    B(t,x,\gamma)_{012}=B(t,x,\gamma)_{s_0l_0,s_1l_1,s_2l_2} = b(t,x)_{l_0l_1l_2}\gamma_{s_0s_1s_2}
$$

In [34]:
for d in cmat.correlators[0].diagrams:
    d.create_hadron_blocks()
    d.combine_indices()

for d in cmat.correlators[0].diagrams:
    pprint(d)

<IPython.core.display.Math object>

Then we finalize transforming the diagrams by combining the perambulators with the sink hadron blocks

$$
    H(t_f,t_i,x,\gamma) = H(t_f,x,\gamma)\tau(t_f | t_i)
$$

In [35]:
for d in cmat.correlators[0].diagrams:
    d.create_hadron_source()

for d in cmat.correlators[0].diagrams:
    pprint(d)

<IPython.core.display.Math object>